In [ ]:
import gin
import tensorflow as tf
from tf_agents.trajectories import trajectory
from compiler_opt.rl import agent_config, registry

data_path = '/home/student/default_trace'

@gin.configurable
def train_eval(agent_config_type=agent_config.BCAgentConfig,
               num_iterations=100,
               batch_size=64,
               train_sequence_length=1):
    pass

gconfig = ["compiler_opt/rl/inlining/gin_configs/behavioral_cloning_nn_agent.gin"]
gbindings = []
gin.parse_config_files_and_bindings(
    gconfig, gbindings, skip_unknown=False
)

In [ ]:
tf.data.Dataset.list_files(data_path)

In [ ]:
tf.data.Dataset.list_files(data_path)\
    .shuffle(100)\
    .filter(lambda string: tf.strings.length(string) > 0)\
    .apply(tf.data.experimental.shuffle_and_repeat(1024))\
    #.filter(lambda traj: tf.size(traj.reward) > 2)

In [ ]:
problem_config = registry.get_configuration()
time_step_spec, action_spec = problem_config.get_signature_spec()

agent_cfg = agent_config.BCAgentConfig(
    time_step_spec=time_step_spec,
    action_spec=action_spec
)

In [ ]:
def create_parser_fn(
    agent_cfg: agent_config.AgentConfig
):
  """Create a parser function for reading from a serialized tf.SequenceExample.

  Args:
    agent_name: AgentName, enum type of the agent.
    time_step_spec: time step spec of the optimization problem.
    action_spec: action spec of the optimization problem.

  Returns:
    A callable that takes scalar serialized proto Tensors and emits
    `Trajectory` objects containing parsed tensors.
  """

  def _parser_fn(serialized_proto):
    """Helper function that is returned by create_`parser_fn`."""
    # We copy through all context features at each frame, so even though we know
    # they don't change from frame to frame, they are still sequence features
    # and stored in the feature list.
    context_features = {}
    # pylint: disable=g-complex-comprehension
    sequence_features = dict(
        (tensor_spec.name,
         tf.io.FixedLenSequenceFeature(
             shape=tensor_spec.shape, dtype=tensor_spec.dtype))
        for tensor_spec in agent_cfg.time_step_spec.observation.values())
    sequence_features[
        agent_cfg.action_spec.name] = tf.io.FixedLenSequenceFeature(
            shape=agent_cfg.action_spec.shape,
            dtype=agent_cfg.action_spec.dtype)
    sequence_features[
        agent_cfg.time_step_spec.reward.name] = tf.io.FixedLenSequenceFeature(
            shape=agent_cfg.time_step_spec.reward.shape,
            dtype=agent_cfg.time_step_spec.reward.dtype)
    sequence_features.update(agent_cfg.get_policy_info_parsing_dict())

    # pylint: enable=g-complex-comprehension
    with tf.name_scope('parse'):
      _, parsed_sequence = tf.io.parse_single_sequence_example(
          serialized_proto,
          context_features=context_features,
          sequence_features=sequence_features)
      # TODO(yundi): make the transformed reward configurable.
      action = parsed_sequence[agent_cfg.action_spec.name]
      reward = tf.cast(parsed_sequence[agent_cfg.time_step_spec.reward.name],
                       tf.float32)

      policy_info = agent_cfg.process_parsed_sequence_and_get_policy_info(
          parsed_sequence)

      del parsed_sequence[agent_cfg.time_step_spec.reward.name]
      del parsed_sequence[agent_cfg.action_spec.name]
      full_trajectory = trajectory.from_episode(
          observation=parsed_sequence,
          action=action,
          policy_info=policy_info,
          reward=reward)
      return full_trajectory

  return _parser_fn


In [ ]:
dict(
        (tensor_spec.name,
         tf.io.FixedLenSequenceFeature(
             shape=tensor_spec.shape, dtype=tensor_spec.dtype))
        for tensor_spec in agent_cfg.time_step_spec.observation.values())

In [ ]:
tf.io.FixedLenSequenceFeature(
            shape=agent_cfg.action_spec.shape,
            dtype=agent_cfg.action_spec.dtype)

In [ ]:
agent_cfg.get_policy_info_parsing_dict()

In [ ]:
parser_fn = create_parser_fn(agent_cfg)

In [ ]:
tf.data.Dataset.list_files(data_path)\
    .shuffle(100)\
    .interleave(tf.data.TFRecordDataset, cycle_length=10, block_length=1)\
    .filter(lambda string: tf.strings.length(string) > 0)\
    .apply(tf.data.experimental.shuffle_and_repeat(1024))

In [ ]:
foo = tf.data.Dataset.list_files(data_path)\
    .shuffle(100)\
    .interleave(tf.data.TFRecordDataset, cycle_length=10, block_length=1)\
    .filter(lambda string: tf.strings.length(string) > 0)\
    .apply(tf.data.experimental.shuffle_and_repeat(1024))

_iter = iter(foo)
for x in range(100):
    next(_iter)
    print('.', end='')
print()

In [ ]:
proto = next(_iter)

In [ ]:
context_features = {}
# pylint: disable=g-complex-comprehension
sequence_features = dict(
    (tensor_spec.name,
     tf.io.FixedLenSequenceFeature(
         shape=tensor_spec.shape, dtype=tensor_spec.dtype))
    for tensor_spec in agent_cfg.time_step_spec.observation.values())
sequence_features[
    agent_cfg.action_spec.name] = tf.io.FixedLenSequenceFeature(
        shape=agent_cfg.action_spec.shape,
        dtype=agent_cfg.action_spec.dtype)
sequence_features[
    agent_cfg.time_step_spec.reward.name] = tf.io.FixedLenSequenceFeature(
        shape=agent_cfg.time_step_spec.reward.shape,
        dtype=agent_cfg.time_step_spec.reward.dtype)
sequence_features.update(agent_cfg.get_policy_info_parsing_dict())

#sequence_features

In [ ]:
# pylint: enable=g-complex-comprehension
with tf.name_scope('parse'):
  _, parsed_sequence = tf.io.parse_single_sequence_example(
      proto,
      context_features=context_features,
      sequence_features=sequence_features)
  # TODO(yundi): make the transformed reward configurable.
  action = parsed_sequence[agent_cfg.action_spec.name]
  reward = tf.cast(parsed_sequence[agent_cfg.time_step_spec.reward.name],
                   tf.float32)

  policy_info = agent_cfg.process_parsed_sequence_and_get_policy_info(
      parsed_sequence)

  del parsed_sequence[agent_cfg.time_step_spec.reward.name]
  del parsed_sequence[agent_cfg.action_spec.name]
  full_trajectory = trajectory.from_episode(
      observation=parsed_sequence,
      action=action,
      policy_info=policy_info,
      reward=reward)

In [ ]:
full_trajectory.reward

In [ ]:
cnt = 0
while tf.size(full_trajectory.reward) <= 2 and cnt < 1024:
    print('.', end='')
    proto = next(_iter)
    # pylint: enable=g-complex-comprehension
    with tf.name_scope('parse'):
      _, parsed_sequence = tf.io.parse_single_sequence_example(
          proto,
          context_features=context_features,
          sequence_features=sequence_features)
      # TODO(yundi): make the transformed reward configurable.
      action = parsed_sequence[agent_cfg.action_spec.name]
      reward = tf.cast(parsed_sequence[agent_cfg.time_step_spec.reward.name],
                       tf.float32)
    
      policy_info = agent_cfg.process_parsed_sequence_and_get_policy_info(
          parsed_sequence)
    
      del parsed_sequence[agent_cfg.time_step_spec.reward.name]
      del parsed_sequence[agent_cfg.action_spec.name]
      full_trajectory = trajectory.from_episode(
          observation=parsed_sequence,
          action=action,
          policy_info=policy_info,
          reward=reward)
      cnt += 1
print()

if tf.size(full_trajectory.reward) > 2:
    print("Found one!")
else:
    print("No luck buckaroo")

Theory: Infinite dataset, so it'll just keep looping until it finds a sample that is valid but it never does b/c there are no with size of reward > 2

In [ ]:
proto = next(_iter)
# pylint: enable=g-complex-comprehension
with tf.name_scope('parse'):
  _, parsed_sequence = tf.io.parse_single_sequence_example(
      proto,
      context_features=context_features,
      sequence_features=sequence_features)
  # TODO(yundi): make the transformed reward configurable.
  action = parsed_sequence[agent_cfg.action_spec.name]
  reward = tf.cast(parsed_sequence[agent_cfg.time_step_spec.reward.name],
                   tf.float32)

  policy_info = agent_cfg.process_parsed_sequence_and_get_policy_info(
      parsed_sequence)

In [ ]:
action

In [ ]:
reward

In [ ]:
policy_info

In [ ]:
agent_cfg.time_step_spec.reward.name

In [ ]:
full_trajectory = trajectory.from_episode(
  observation=parsed_sequence,
  action=action,
  policy_info=policy_info,
  reward=reward)

In [ ]:
full_trajectory

---

In [ ]:
d = tf.data.Dataset.list_files(data_path)\
    .shuffle(100)\
    .interleave(tf.data.TFRecordDataset, cycle_length=10, block_length=1)\
    .filter(lambda string: tf.strings.length(string) > 0)\
    .apply(tf.data.experimental.shuffle_and_repeat(1024))\
    .map(parser_fn, num_parallel_calls=8)\
    #.filter(lambda traj: tf.size(traj.reward) > 2)

In [ ]:
d

In [ ]:
tf.data.Dataset.cardinality(d)

In [ ]:
d2 = (d.unbatch()\
      .batch(
        16, # arbitrary number for now
        drop_remainder=True)\
      .shuffle(1024)\
      .batch(16, drop_remainder=True)\
      .prefetch(tf.data.experimental.AUTOTUNE))

In [ ]:
d2

This next cell is the killer :(

In [ ]:
next(iter(d2))